This notebook was written on colab. It requires CUDA and CUDNN. This notebook was used to train YOLOV3 on the colour-helmet dataset using pjreddie's darknet repository. 

In [ ]:
# Simply for checking cuda and cudnn installations

!nvidia-smi
!nvcc --version
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

Wed Jul  6 03:24:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# IMPORT DARKNET

In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5952, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5952 (delta 2), reused 0 (delta 0), pack-reused 5946
Receiving objects: 100% (5952/5952), 6.37 MiB | 30.77 MiB/s, done.
Resolving deltas: 100% (3930/3930), done.


## Make darknet build changes

In [ ]:
%cd /content/darknet

%pycat Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

%pycat Makefile

#Comment out the compute 30 arch line in MAKE file before building
#In examples/detection.c change the weights save freq(in line 138)

/content/darknet


## Build darknet and check build

In [ ]:
%cd /content/darknet
!make
!./darknet

/content/darknet
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:1004:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -

## Dowload model from drive

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/Computer_Vision/Yolo_builds/Colour_Helmet_detection', '/content/darknet/Yolo_builds/Colour_Helmet_detection')

## Unzip the data

In [ ]:
%cd /content/darknet/Yolo_builds/Colour_Helmet_detection
!unzip Data_set/Images_and_annotations.zip -d Data_set/Images_and_annotations

# Make the neccessary .conf and .data model file changes now

In [ ]:
%cd /content/darknet/Yolo_builds/Colour_Helmet_detection

!sudo apt install dos2unix
!dos2unix train_data.txt
!dos2unix valid_data.txt
!dos2unix Colour_Helmet_detection.data
!dos2unix class_names.txt
!dos2unix yolov3-voc.cfg

/content/darknet/Yolo_builds/Colour_Helmet_detection
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 0s (1,690 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
d

# TRAIN

In [ ]:
%cd /content/darknet
!./darknet detector train Yolo_builds/Colour_Helmet_detection/Colour_Helmet_detection.data Yolo_builds/Colour_Helmet_detection/yolov3-voc.cfg /content/drive/MyDrive/Computer_Vision/Yolo_builds/Colour_Helmet_detection/Weights_save/yolov3-voc_9000.weights

#/content/drive/MyDrive/Computer_Vision/Yolo_builds/Colour_Helmet_detection/Weights_save/yolov3-voc_400.weights


"""
classes= 3
backup = /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save
names = /content/darknet/Yolo_builds/Helmet_detection/class_names.txt
valid  = /content/darknet/Yolo_builds/Helmet_detection/valid_data.txt
train  = /content/darknet/Yolo_builds/Helmet_detection/train_data.txt


learning_rate=0.000003
burn_in=1
max_batches = 15000
policy=steps
steps=14000
scales=.1,.1

# 6, 8, 11, 16, 18, 26, 25, 37, 33, 52, 46, 68, 59, 92, 81, 118, 118, 160
#12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401

"""
#0.0001 
#0.00003 @ 500
#0.00001 @ 1300

/content/darknet
yolov3-voc
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res

'\nclasses= 3\nbackup = /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save\nnames = /content/darknet/Yolo_builds/Helmet_detection/class_names.txt\nvalid  = /content/darknet/Yolo_builds/Helmet_detection/valid_data.txt\ntrain  = /content/darknet/Yolo_builds/Helmet_detection/train_data.txt\n\n\nlearning_rate=0.000003\nburn_in=1\nmax_batches = 15000\npolicy=steps\nsteps=14000\nscales=.1,.1\n\n\n\n'